In [1]:
import glob
import os
from os.path import join

import pandas as pd

from modl.datasets import get_data_dirs


def fetch_archi(data_dir=None, n_subjects=None):
    data_dir = get_data_dirs(data_dir)[0]
    source_dir = join(data_dir, 'archi', 'unsmoothed')
    if not os.path.exists(source_dir):
        raise ValueError('Please ensure that archi can be found under $MODL_DATA'
                         'repository.')
    z_maps = glob.glob(join(source_dir, '*', '*_z_map.nii'))
    subjects = []
    contrasts = []
    for z_map in z_maps:
        dirname, filename = os.path.split(z_map)
        _, subject = os.path.split(dirname)
        subject = int(subject[-3:])
        contrast = filename.replace('_z_map.nii', '')
        subjects.append(subject)
        contrasts.append(contrast)
    df = pd.DataFrame(data=[pd.Series(data=subjects, name='subject'),
                            pd.Series(data=contrasts, name='contrast'),
                            pd.Series(data=z_maps, name='z_map')]).T
    df.set_index(['subject', 'contrast'], inplace=True)
    df.sort_index(inplace=True)
    subjects = df.index.get_level_values('subject').unique().values.tolist()
    df = df.loc[subjects[:n_subjects]]
    return df

In [2]:
df = fetch_archi(n_subjects=10)

In [3]:
df.index.get_level_values('subject').unique()

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 10, 11], dtype='int64', name='subject')

In [4]:
contrasts = df.index.get_level_values('contrast').unique().values

In [5]:
import re
prog = re.compile(r"^[^-]*\Z")

In [6]:
interesting_contrasts = []
for contrast in contrasts:
    if prog.match(contrast):
        interesting_contrasts.append(contrast)

In [7]:
interesting_contrasts

['all_expression',
 'all_face',
 'effects_of_interest',
 'expression_control',
 'expression_intention',
 'expression_sex',
 'face_control',
 'face_sex',
 'face_trusty',
 'false_belief_audio',
 'false_belief_video',
 'mecanistic_audio',
 'mecanistic_video',
 'non_speech',
 'object_grasp',
 'object_orientation',
 'rotation_hand',
 'rotation_side',
 'saccade',
 'speech',
 'triangle_intention',
 'triangle_random']

In [143]:
contrast[0]

'H-V'